In [5]:
"""
Format Dataset: (list of lists) 
    [
        [context_0, context_1, ..., context_N, response], # sample_0
        [context_0, context_1, ..., context_N, response], # sample_1
        ...,
        [context_0, context_1, ..., context_N, response] # sample_L
    ]
"""

'\nFormat Dataset: (list of lists) \n    [\n        [context_0, context_1, ..., context_N, response], # sample_0\n        [context_0, context_1, ..., context_N, response], # sample_1\n        ...,\n        [context_0, context_1, ..., context_N, response] # sample_L\n    ]\n'

In [6]:
import pandas as pd
# tokenizer
from transformers import AutoTokenizer
import config as cfg

In [7]:
# Parameters
# concatenate N_PAIRS+1 pairs of sentences (user1: txt, user2: text), the last sentence would be a sentence from the interest user
N_PAIRS = 7 # use 7 sentences like input (context) and the next sentence like a target
TEST_SIZE = 0.1


# Paths
#"Dataset/Yoda/yoda-corpus.csv"
#"Dataset/Rick/RickAndMortyScripts.csv"
PATH_DATASET = "Dataset/Rick/RickAndMortyScripts.csv"

# Instance Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained(cfg.output_dir)



In [8]:
df = pd.read_csv(PATH_DATASET)

# Prepare datset setting
## the dataframe should be to have this columns ----> [name, line]
## Define name of character to emulate hsi personality -----> characheter_principal_name 

print(len(df))
df.head(20)

1905


,index,season no.,episode no.,episode name,name,line
0,0,1,1,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,1,1,1,Pilot,Morty,"What, Rick? What’s going on?"
2,2,1,1,Pilot,Rick,"I got a surprise for you, Morty."
3,3,1,1,Pilot,Morty,It's the middle of the night. What are you tal...
4,4,1,1,Pilot,Rick,"Come on, I got a surprise for you. Come on, h..."
5,5,1,1,Pilot,Morty,Ow! Ow! You're tugging me too hard!
6,6,1,1,Pilot,Rick,"We gotta go, gotta get outta here, come on. Go..."
7,7,1,1,Pilot,Rick,"What do you think of this... flying vehicle, M..."
8,8,1,1,Pilot,Morty,"Yeah, Rick... I-it's great. Is this the surprise?"
9,9,1,1,Pilot,Rick,Morty. I had to... I had to do it. I had— I ha...


# Format data
## add special tokens

In [21]:
MIN_SENTENCES = 7
CHARACTER_PRINCIPAL_NAME = 'Rick'

# identify all id sentences of the principañ character
id_principal_character_speak = df[df.name==CHARACTER_PRINCIPAL_NAME].index.tolist()
Samples = []

#ix = id_principal_character_speak[5]
for ix in id_principal_character_speak:
    if ix-MIN_SENTENCES<0:
        #ix_ant=None
        continue
    else:
        ix_ant = ix-MIN_SENTENCES


    Line=[]
    for i in range(ix_ant,ix+1):
        line = df.line[i]
        name = df.name[i]

        # identify if the character that is specaking is the bot or a user
        name = cfg.bot_token if name==CHARACTER_PRINCIPAL_NAME else cfg.user_token
        line = name + ' ' + line

        # concatenate sentences
        Line.append(line)
    
        #sample = cfg.bos_token +' ' + ' '.join(Line) + ' ' + cfg.eos_token
    Samples.append(Line)

In [22]:
columns = [f"context_{i}" for i in range(MIN_SENTENCES)] + ['response']
df_prep = pd.DataFrame(Samples,columns=[columns])
df_prep.head()

,context_0,context_1,context_2,context_3,context_4,context_5,context_6,response
0,<|bot|> Morty! You gotta come on. Jus'... you ...,"<|user|> What, Rick? What’s going on?","<|bot|> I got a surprise for you, Morty.",<|user|> It's the middle of the night. What ar...,"<|bot|> Come on, I got a surprise for you. Co...",<|user|> Ow! Ow! You're tugging me too hard!,"<|bot|> We gotta go, gotta get outta here, com...",<|bot|> What do you think of this... flying ve...
1,"<|bot|> I got a surprise for you, Morty.",<|user|> It's the middle of the night. What ar...,"<|bot|> Come on, I got a surprise for you. Co...",<|user|> Ow! Ow! You're tugging me too hard!,"<|bot|> We gotta go, gotta get outta here, com...",<|bot|> What do you think of this... flying ve...,"<|user|> Yeah, Rick... I-it's great. Is this t...",<|bot|> Morty. I had to... I had to do it. I h...
2,"<|bot|> Come on, I got a surprise for you. Co...",<|user|> Ow! Ow! You're tugging me too hard!,"<|bot|> We gotta go, gotta get outta here, com...",<|bot|> What do you think of this... flying ve...,"<|user|> Yeah, Rick... I-it's great. Is this t...",<|bot|> Morty. I had to... I had to do it. I h...,<|user|> What?! A bomb?!,<|bot|> We're gonna drop it down there just ge...
3,"<|bot|> We gotta go, gotta get outta here, com...",<|bot|> What do you think of this... flying ve...,"<|user|> Yeah, Rick... I-it's great. Is this t...",<|bot|> Morty. I had to... I had to do it. I h...,<|user|> What?! A bomb?!,<|bot|> We're gonna drop it down there just ge...,<|user|> T-t-that's absolutely crazy!,"<|bot|> Come on, Morty. Just take it easy, Mor..."
4,"<|user|> Yeah, Rick... I-it's great. Is this t...",<|bot|> Morty. I had to... I had to do it. I h...,<|user|> What?! A bomb?!,<|bot|> We're gonna drop it down there just ge...,<|user|> T-t-that's absolutely crazy!,"<|bot|> Come on, Morty. Just take it easy, Mor...",<|user|> Jessica? From my math class?,"<|bot|> When I drop the bomb you know, I want ..."


In [23]:
# save
df_prep.to_csv("Rick_dataset_preproced.csv", index=False)